# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверьте четыре гипотезы:
1. Вероятность погашения кредита в срок зависит от наличия детей.
2. Семейное положение оказывает влияния на вероятность погашения кредита в срок.
3. Вероятность погашения кредита в срок зависит от уровня дохода.
4. На вероятность погашения кредита в срок оказывает влияние цель кредита.

**Ход исследования**

Исследование пройдёт в четыре этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез и ответы на вопросы.
 4. Выводы.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Импортируем библиотеки, которые потребуются для реализации проекта

In [1]:
# импорт библиотеки pandas
import pandas as pd

# импорт библиотеки для лемматизации
from pymystem3 import Mystem
m = Mystem() 

# импорт библиотеки для подсчета числа лемм
from collections import Counter

Прочтем файл со статистикой о платёжеспособности клиентов и рассмотрим общую информацию о датасете

In [2]:
# чтение файла с данными и сохранение в df
data = pd.read_csv('/datasets/data.csv')

Выведем первые 10 строк датасета

In [3]:
# получение первых 10 строк датасета
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем общую информацию о датасете

In [4]:
# получение общей информации о данных датасета
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Выведем описательную статистику

In [5]:
# получение описательной статистики данных датасета
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак, в таблице 12 столбцов и 21525 строк. В столбцах содержатся различные типы данных — `float`, `int` и `object`.

Описание данных:
* `children` — количество детей в семье (тип данных `int`, заполнены все строки);
* `days_employed` — трудовой стаж в днях (тип данных `float`, в некоторых строках данные отсутствуют);
* `dob_years` — возраст клиента в годах (тип данных `int`, заполнены все строки);
* `education` — образование клиента (тип данных `object`, заполнены все строки);
* `education_id` — идентификатор образования (тип данных `int`, заполнены все строки);
* `family_status` — семейное положение (тип данных `object`, заполнены все строки);
* `family_status_id` — идентификатор семейного положения (тип данных `int`, заполнены все строки);
* `gender` — пол клиента (тип данных `object`, заполнены все строки);
* `income_type` — тип занятости (тип данных `object`, заполнены все строки);
* `debt` — имел ли задолженность по возврату кредитов: 1 - имел, 0 - нет (тип данных `int`, заполнены все строки);
* `total_income` — доход в месяц (тип данных `float`, в некоторых строках данные отсутствуют);
* `purpose` — цель получения кредита (тип данных `object`, заполнены все строки).

**Вывод**

Каждая строка таблицы содержит данные о клиенте и цели кредита. Большая часть столбцов рассказывает о клиенте: количество детей, трудовой стаж, возраст, образование, семейное положение, пол, тип занятости, информация о задолженностях и доход. Последний столбец описывает цель кредита. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез, названия колонок заданы согласно хорошему стилю и не требуют изменений. Но встречаются пропуски в данных столбцов `days_employed` и `total_income`.

В столбце `children` присутствуют отлицательные значения, а также значение 20, которое лучше проверить.

Столбец `days_employed` представлен в днях, но имеет тип float, а также имеет отрицательные значения и значения, которые превышают разумно возможные значения стажа. После перевода стажа из дней в года данные будут нагляднее.

Минимальное значение в столбце `dob_years` - 0, что является аномальным значением.

Столбец `education` содержит записи разного регистра.

Столбец `total_income` имеет тип данных float. Несмотря на отсутствие единицы измерения, считаю что перевод его в тип int не повлияет на результаты анализа.

Столбец `purpose` имеет одни и те же цели, обозначенные разными словами.

В датасете присутствует избыточность данных. Для проверки гипотез нас интересуют столбцы `children`, `family_status`, `debt`, `total_income` и `purpose`. Остальные столбцы можно исключить из рассмотрения.

Необходимо устранить следующие проблемы в данных:
1. Обработать пропуски в столбцах `days_employed` и `total_income`.
2. Перевести данные о стаже из дней в года в столбце `days_employed`, перемеиновать его в `years_employed` и заменить вещественный тип данных на целочисленный в столбцах `years_employed` и `total_income`.
3. Устранить аномалии в данных.
4. Найти и устранить дубликаты.

Для проверки гипотез можно ограничиться таблицей с необходимыми данными, которые содержат столбцы `children`, `family_status`, `debt`, `total_income` и `purpose`.

*P.S. Для проверки гипотез можно сразу исключить избыточные данные и работать только с необходимыми, но в целях обучения проведем предобработку всех данных.*

## Шаг 2. Предобработка данных

### Обработка пропусков

Пропуски присутствуют в двух столбцах `days_employed` и `total_income`. Посчитаем суммарное количество пропусков.

In [6]:
# подсчёт пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбцы с данными о стаже и доходах содержат равное количество пропусков, которое составляет примерно 10% всего датасета. Вероятно это информация о клиентах, которые не работали официально и не имели официального дохода. Пропуски в данных о стаже в данном исследовании не важны, а пропуски в доходах могут оказать значительное влияние.
Удалим строки с пропусками, т.к. присваивание любых значений окажет непредсказуемое влияние на результат.

In [7]:
# удаление строк с пропусками в доходе
data = data.dropna(subset=['total_income'])

Убедимся, что в датасете не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [8]:
# подсчёт пропусков
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Получим описательную статистику и оценим изменения данных после удаления строк.

In [9]:
# получение описательной статистики данных датасета
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,1.935100e+04
mean,0.537388,63046.497661,43.255336,0.819079,0.972249,0.081184,1.674223e+05
std,1.371408,140827.311974,12.579170,0.550104,1.420596,0.273125,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Были удалены строки с пропусками, их количество составило примерно 10% от всего датасета. Изучив средние значения в столбцах, можно прийти к выводу, что значения изменились менее чем на 1%, что нас полностью устраивает.

### Замена типа данных

Переведем данные о стаже из дней в года в столбце `days_employed` и переменуем его в `years_employed`.

In [10]:
# перевод стажа в года и переименование
data['years_employed'] = data['days_employed'] / 365

Заменим тип данных с вещественного на целочисленный и избавимся от лишнего столбца с данными о стаже в днях.

In [11]:
# замена типа данных
data['years_employed'] = data['years_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# удаление столбца
del data['days_employed']

# избавление от отрицательных значений
data['years_employed'] = data['years_employed'].abs()

In [12]:
# получение первых 10 строк датасета
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,23
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,11
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,15
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,11
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,932
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,2
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,7
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование,0
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,18
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,5


Столбец со стажем явно имеет аномальные значения, но для проверки гипотез он не используется, так что оставим все как есть.

**Вывод**

В столбцах со стажем и доходом был изменен тип данных с вещественного на целочисленный. Для этого был использован метод `astype()`, так как он переводит данные в нужный нам целочисленный тип. Осталось много аномалий, с которыми надо разобраться.

### Устранение аномалий в данных

Избавимся от них сначала в столбце `children`. Для этого изучим уникальные значения.

In [13]:
# получение уникальных значений 
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Значение -1 явно ошибочно, а наличие 20 детей хотя и маловероятно, но все же возможно. Проверим количество уникальных значений, чтобы сделать вывод о 20 детях.

In [14]:
# получение уникальных значений и их количества
data['children'].value_counts()

 0     12710
 1      4343
 2      1851
 3       294
 20       67
-1        44
 4        34
 5         8
Name: children, dtype: int64

Наличие 67 клиентов с 20 детьми маловероятно, скорее всего это случайная ошибка заполнения данных. Заменим 20 на 2.

Аналогично -1 - случайная ошибка при написании 1.

In [15]:
# замена значения
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

Проверим уникальные значения.

In [16]:
# проверка уникальных значений
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

С детьми разобрались, проверим другие столбцы.

In [17]:
# получение уникальных значений и их количества
data['dob_years'].value_counts()

35    553
41    548
38    544
40    543
34    534
42    532
33    530
39    522
44    503
31    495
29    495
48    492
36    492
37    484
30    482
32    473
43    463
50    463
49    458
27    457
45    447
28    446
56    433
52    431
46    427
54    424
47    421
53    415
59    410
58    405
57    404
51    398
55    395
26    373
60    338
25    334
61    317
62    314
24    243
63    240
64    228
23    218
65    174
22    166
66    163
67    151
21     93
0      91
68     90
69     80
70     62
71     53
20     46
72     31
19     13
73      7
74      6
75      1
Name: dob_years, dtype: int64

Вызывает удивление наличие 91 клиента в возрасте 0. К счастью эти данные не участвуют в проверке гипотез, поэтому мы можем ими пренебречь.

In [18]:
# получение уникальных значений и их количества
data['education'].value_counts()

среднее                12342
высшее                  4222
СРЕДНЕЕ                  705
Среднее                  646
неоконченное высшее      613
ВЫСШЕЕ                   251
Высшее                   243
начальное                231
Неоконченное высшее       40
НЕОКОНЧЕННОЕ ВЫСШЕЕ       22
НАЧАЛЬНОЕ                 16
Начальное                 14
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Разнообразие написание одних и тех же значений велико. Приведем все к единому стилю и проверим уникальные значения.

In [19]:
# приведение к нижнему регистру
data['education'] = data['education'].str.lower()

# проверка уникальных значений
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [20]:
# проверка уникальных значений
data['education_id'].unique()

array([0, 1, 2, 3, 4])

Количество категорий в обоих столбцах совпадает.

In [21]:
# получение уникальных значений и их количества
data['family_status'].value_counts()

женат / замужем          11143
гражданский брак          3735
Не женат / не замужем     2525
в разводе                 1083
вдовец / вдова             865
Name: family_status, dtype: int64

In [22]:
# проверка уникальных значений
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

Количество категорий в обоих столбцах совпадает. В этих столбцах проблем нет.

In [23]:
# получение уникальных значений и их количества
data['gender'].value_counts()

F      12752
M       6598
XNA        1
Name: gender, dtype: int64

Присутствует странное значение, но примем допущение, что банк европейский, а Европа - такая Европа =)

In [24]:
# получение уникальных значений и их количества
data['income_type'].value_counts()

сотрудник          10014
компаньон           4577
пенсионер           3443
госслужащий         1312
безработный            2
предприниматель        1
в декрете              1
студент                1
Name: income_type, dtype: int64

In [25]:
# получение уникальных значений и их количества
data['debt'].value_counts()

0    17780
1     1571
Name: debt, dtype: int64

Со столбцом с целями разберемся чуть позже.

**Вывод**

Устранены выявленные аномалии в данных, такие как выбросы, стиль написания и отрицательные значения. 

### Обработка дубликатов

Проверим датасет на наличие явных дубликатов.

In [26]:
# подсчёт явных дубликатов
data.duplicated().sum()

0

Поищем неявные дубликаты. Без проверки остался лишь столбец с целью кредита. Отсортируем его и выведем цели на экран.

In [27]:
# получение уникальных значений
sorted_purposes = sorted(data['purpose'].unique())
sorted_purposes

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

Невооруженным взглядом можно обнаружить, что некоторые цели дублируют друг друга. Это произошло, потому что не хватает заранее определенных формулировок целей из которых бы делался выбор.

**Вывод**

Явные дубликаты отсутствуют, а вот неявные есть и их нужно устранить.

### Лемматизация

Лемматизация — процесс приведения словоформы к лемме — её нормальной (словарной) форме. Используем библиотеку с функцией лемматизации на русском языке — pymystem3, которая была разработана сотрудниками Яндекса. С помощью лемматизации найдем дубликаты.

In [28]:
# перевод уникальных значений в строку
credit_purposes = data['purpose'].unique().tolist()
string = ', '.join(credit_purposes)

# лемматизация строки
lemmas = m.lemmatize(string)

# подсчет числа лемм
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ', ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'получение': 3,
         'сделка': 2,
         'подержанный': 1,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Из данных можно сделать выводы, что существует четыре обобщенных целей кредитов:

1. покупка автомобиля
2. получение образования
3. покупка недвижимости
4. организация свадьбы

Избавимся от неявых дубликатов. Для этого с помощью циклов создадим списки со схожими целями.

In [29]:
# поиск наименований целей, связанных с автомобилем
car = []
for phrase in sorted_purposes:
    if 'автомоб' in phrase:
        car.append(phrase)
car

['автомобили',
 'автомобиль',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'приобретение автомобиля',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем']

In [30]:
# поиск наименований целей, связанных с образованием
education = []
for phrase in sorted_purposes:
    if 'образов' in phrase:
        education.append(phrase)
education

['высшее образование',
 'дополнительное образование',
 'заняться высшим образованием',
 'заняться образованием',
 'образование',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'профильное образование']

In [31]:
# поиск наименований целей, связанных с недвижимостью
real_estate = []
for phrase in sorted_purposes:
    if 'жил' in phrase:
        real_estate.append(phrase)
    if 'недвиж' in phrase:
        real_estate.append(phrase)
real_estate

['жилье',
 'недвижимость',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'ремонт жилью',
 'строительство жилой недвижимости',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недвижимости']

In [32]:
# поиск наименований целей, связанных со свадьбой
wedding = []
for phrase in sorted_purposes:
    if 'свадь' in phrase:
        wedding.append(phrase)
wedding

['на проведение свадьбы', 'свадьба', 'сыграть свадьбу']

Напишем функцию, с помощью которой приведем категоризацию целей. 

In [33]:
# Функция для замены неявных дубликатов
def replace_wrong_purposes(wrong_purposes, correct_purposes):
    for wrong_purpose in wrong_purposes:
        data['purpose'] = data['purpose'].replace(wrong_purposes, correct_purposes)
        
correct_name_car = 'покупка автомобиля'
correct_name_education = 'получение образования'
correct_name_real_estate = 'покупка недвижимости'
correct_name_wedding = 'организация свадьбы'

In [34]:
# Устранение неявных дубликатов
replace_wrong_purposes(car, correct_name_car)
replace_wrong_purposes(education, correct_name_education)
replace_wrong_purposes(real_estate, correct_name_real_estate)
replace_wrong_purposes(wedding, correct_name_wedding)

In [35]:
# Проверка на неявные дубликаты
data['purpose'].unique()

array(['покупка недвижимости', 'покупка автомобиля',
       'получение образования', 'организация свадьбы'], dtype=object)

Проверка демонстрирует отсутствие неявных дубликатов.

**Вывод**

С помощью лемматизации были выявлены неявные дубликаты, переименованы цели и тем самым категоризированы. 

### Категоризация данных

Столбец с данными о доходах имеет большое количество данных, присутствующих в наборе лишь единожды. Работать с такими данными и делать из них статистические выводы нельзя. Поэтому следует категоризировать такие данные, т.е. объединение данных в категории.

In [36]:
# оценка значений квантилей
data['total_income'].quantile([0.25,0.5,0.75])

0.25    103053.0
0.50    145017.0
0.75    203434.5
Name: total_income, dtype: float64

Распределим уровень дохода на 4 группы и присвоим каждой группе статусы. Разделение проведем по полученным значениям квантилей.

In [37]:
# описание функции для категоризации доходов
def income_status(total_income):
    if total_income <= 103053.0:
            return 'Низкий уровень дохода'
    elif total_income <= 145017.0:
            return 'Средний уровень дохода'
    elif total_income < 203434.5:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'

In [38]:
# присвоение статусов
data['income_status'] = data['total_income'].apply(income_status)

In [39]:
# получение первых 10 строк датасета
data.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,income_status
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка недвижимости,23,Сверхвысокий уровень дохода
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,покупка автомобиля,11,Средний уровень дохода
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка недвижимости,15,Высокий уровень дохода
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,получение образования,11,Сверхвысокий уровень дохода
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,организация свадьбы,932,Высокий уровень дохода
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка недвижимости,2,Сверхвысокий уровень дохода
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,покупка недвижимости,7,Сверхвысокий уровень дохода
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,получение образования,0,Средний уровень дохода
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,организация свадьбы,18,Низкий уровень дохода
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка недвижимости,5,Средний уровень дохода


Созданим таблицу с данными, которые потребуются для проверки гипотез.

In [40]:
# Таблица с результатами
df = {'children': data['children'], 
      'family_status': data['family_status'], 
      'income_status': data['income_status'],
      'purpose': data['purpose'],
      'debt': data['debt'],
     }
data_to_hypothesis = pd.DataFrame(df)

In [41]:
# получение последних 10 строк датасета
data_to_hypothesis.tail(10)

,children,family_status,income_status,purpose,debt
21515,1,женат / замужем,Средний уровень дохода,получение образования,1
21516,0,женат / замужем,Сверхвысокий уровень дохода,покупка недвижимости,0
21517,0,гражданский брак,Высокий уровень дохода,покупка автомобиля,0
21518,0,женат / замужем,Высокий уровень дохода,покупка автомобиля,0
21519,1,в разводе,Средний уровень дохода,покупка недвижимости,0
21520,1,гражданский брак,Сверхвысокий уровень дохода,покупка недвижимости,0
21521,0,женат / замужем,Высокий уровень дохода,покупка автомобиля,0
21522,1,гражданский брак,Низкий уровень дохода,покупка недвижимости,1
21523,3,женат / замужем,Сверхвысокий уровень дохода,покупка автомобиля,1
21524,2,женат / замужем,Низкий уровень дохода,покупка автомобиля,0


**Вывод**

Была проведена категоризация дохода, с помощью выделения 4 групп в зависимости от суммы дохода. Создан датасет с данными, которые требуются для проверки гипотез, без использования избыточных данных.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
# сводная таблица для проверки гипотезы
children_pivot = data.pivot_table(index = ['children'], 
                                  columns = ['debt'], 
                                  values = 'purpose', aggfunc='count')
children_pivot['ratio'] = children_pivot[1] / children_pivot[0] * 100
display(children_pivot)

debt,0,1,ratio
children,,,
0,11758.0,952.0,8.096615
1,3978.0,409.0,10.281549
2,1733.0,185.0,10.675130
3,272.0,22.0,8.088235
4,31.0,3.0,9.677419
5,8.0,NaN,NaN


**Вывод**

Судя по данным в среднем не возвращают кредиты 10% клиентов. При этом наличие детей практически не влияет, хотя можно заметить, что наличие детей все же чуть улучшает статистику. Возможно это связано с тем, что при наличии детей люди подходят более ответственно к принятию решения о получении кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
# сводная таблица для проверки гипотезы
family_status_pivot = data.pivot_table(index = ['family_status'], 
                                  columns = ['debt'], 
                                  values = 'purpose', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
display(family_status_pivot)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2271,254,11.184500
в разводе,1007,76,7.547170
вдовец / вдова,809,56,6.922126
гражданский брак,3396,339,9.982332
женат / замужем,10297,846,8.215985


**Вывод**

Самые низкие показатели возврата кредита у людей в разводе и вдов/вдовцов. Скорее всего это связано с тем, что эти клиенты рассчитывали на помощь партнера по погашению кредита, но обстоятельства изменились.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
# сводная таблица для проверки гипотезы
income_status_pivot = data.pivot_table(index = ['income_status'], 
                                  columns = ['debt'], 
                                  values = 'purpose', aggfunc='count')
income_status_pivot['ratio'] = income_status_pivot[1] / income_status_pivot[0] * 100
display(income_status_pivot)

debt,0,1,ratio
income_status,,,
Высокий уровень дохода,4411,426,9.657674
Низкий уровень дохода,4455,383,8.597082
Сверхвысокий уровень дохода,4497,341,7.582833
Средний уровень дохода,4417,421,9.531356


**Вывод**

Клиенты со сверхвысокими и низкими доходами возвращают кредиты хуже. Зависимость с низкими доходами вполне очевидна, а со сверхвысокими, возможно, связана с тем, что такие клиенты просто забывают оплачивать кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
# сводная таблица для проверки гипотезы
purpose_pivot = data.pivot_table(index = ['purpose'], 
                                  columns = ['debt'], 
                                  values = 'income_status', aggfunc='count')
purpose_pivot['ratio'] = purpose_pivot[1] / purpose_pivot[0] * 100
display(purpose_pivot)

debt,0,1,ratio
purpose,,,
организация свадьбы,1941,158,8.140134
покупка автомобиля,3530,367,10.396601
покупка недвижимости,9043,715,7.906668
получение образования,3266,331,10.134721


**Вывод**

Чаще всего проблемы с возвратом кредита имеются у клиентов, которые берут его на организацию свадьбы и покупку недвижимости. Возможно это связано с большим количеством быстрых разводов =)

## Шаг 4. Общий вывод

Есть зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье. Нельзя сказать, что они кардинально отличаются, но 2% на больших числах могут оказать существенное влияние на кредитный портфель банка. Особое внимание следует уделить вдовам/вдовцам или молодоженам — с ними могут риски по невозврату кредитов выше.

Рекомендации для банка
- Улучшить электронные бланки заявок: ввести туда выпадающие списки и категории. Не позволять подавать заявки с пропущенными данными.

- Считать стаж работы в годах, пользователи чаще всего не знают свой стаж в днях.
- Показывать операторам в банке или пользователям на сайте подсказки заранее, когда вводят сверхвысокие/сверхнизкие
  значения в полях: возраст, стаж, количество детей.
- Блокировать отправку заявок с заведомо некорректными значениями, которые могут быть важны для анализа, например,
  с отрицательным количеством детей.